In [ ]:
import numpy as np
from utils.convex_sets import Polyhedron
from pydrake.all import MathematicalProgram, MosekSolver

In this notebook we test the convex relaxation from the paper on two simple bilinear-optimization problems taken from Section IX.1 of Horst and Reiner "Global optimization: Deterministic approaches."
These are all the examples given in the book in which the convex sets are bounded.
In both cases, our convex relaxation is tight.

In [ ]:
def bilinear_program(X, Phi, p, q, r):

    prog = MathematicalProgram()

    phi = prog.NewContinuousVariables(Phi.dimension)
    x = prog.NewContinuousVariables(X.dimension)
    omega = prog.NewContinuousVariables(Phi.dimension * X.dimension)

    Phi.add_membership_constraint(prog, phi)
    X.add_membership_constraint(prog, x)

    for i in range(Phi.C.shape[0]):
        scale = Phi.d[i] - Phi.C[i].dot(phi)
        vector =  Phi.d[i] * x - np.kron(Phi.C[i], np.eye(X.dimension)).dot(omega)
        X.add_perspective_constraint(prog, scale, vector)

    obj = p.dot(phi) + q.dot(x) + r.dot(omega)
    prog.AddLinearCost(- obj)

    solver = MosekSolver()
    result = solver.Solve(prog)
    
    obj = - result.get_optimal_cost()
    phi = result.GetSolution(phi)
    x = result.GetSolution(x)
    omega = result.GetSolution(omega)
    
    return obj, phi, x, omega

- Example from Konno "A cutting plane algorithm for solving bilinear programs" (Figure 4.1). See also Example IX.1 from Horst and Reiner "Global optimization: Deterministic approaches."
- Our convex relaxation is tight: optimal value is 13.

In [ ]:
C1 = np.array([[1, 4], [4, 1], [3, 4], [-1, 0], [0, -1]])
d1 = np.array([8, 12, 12, 0, 0])
C2 = np.array([[2, 1], [1, 2], [1, 1], [-1, 0], [0, -1]])
d2 = np.array([8, 8, 5, 0, 0])
Phi = Polyhedron(ineq_matrices=(C1, d1))
X = Polyhedron(ineq_matrices=(C2, d2))

p = np.array([-1, 1])
q = np.array([1, 0])
r = np.array([1, -1, -1, 1])

obj, phi, x, omega = bilinear_program(X, Phi, p, q, r)
print(obj)

- Example from Gallo and Ulkucu "Bilinear programming: an exact algorithm" (Appendix A). See also Example IX.2 from Horst and Reiner "Global optimization: Deterministic approaches."
- Our convex relaxation is tight: optimal value is 18.

In [ ]:
C1 = np.array([
    [1, 1],
    [2, 1],
    [3, -1],
    [1, -2],
    [-1, 0],
    [0, -1]
])
d1 = np.array([5, 7, 6, 1, 0, 0])
C2 = np.array([
    [1, 2],
    [3, 1],
    [2, 0],
    [0, 1],
    [-1, 0],
    [0, -1]
])
d2 = np.array([8, 14, 9, 3, 0, 0])
Phi = Polyhedron(ineq_matrices=(C1, d1))
X = Polyhedron(ineq_matrices=(C2, d2))

p = np.array([2, 0])
q = np.array([0, 1])
r = np.array([1, -1, -1, 1])

obj, phi, x, omega = bilinear_program(X, Phi, p, q, r)
print(obj)